In [4]:
#---------------------------------------Extract needed columns from both files-----------------
import pandas as pd

df_TA100=pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Documents\WFP Dashboard and Wages Model\Raw Timesheets\Timecard TA100.xlsx", engine='openpyxl')
df_TA=pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Documents\WFP Dashboard and Wages Model\Raw Timesheets\Timecard TA.xlsx", engine='openpyxl')
df_TA100=df_TA100[['exp_3',	'lastname',	'firstname','procdate',	'begacttime','endacttime','catcode','group','reg','ot1','ot2','ot3','fulltime',	'permanent']]
df_TA=df_TA[['exp_3',	'lastname',	'firstname','procdate',	'begacttime','endacttime','catcode','group','reg','ot1','ot2','ot3','fulltime',	'permanent']]

#combine both TA100 and TA timesheets
df= pd.concat([df_TA100,df_TA], ignore_index=True)
df=df.rename(columns={'exp_3': 'ID',
             'procdate':'Date',
             'begacttime':'Start Time',
             'endacttime':'End Time'})

#-----Update workcode, absence and group
df_absent=pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Documents\WFP Dashboard and Wages Model\Raw Timesheets\CALCULATION.xlsx", engine='openpyxl',sheet_name='Absence', usecols=['catcode','Work Code','Absent?'])
df_group=pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Documents\WFP Dashboard and Wages Model\Raw Timesheets\CALCULATION.xlsx", engine='openpyxl',sheet_name='Group')
# Update absence
df=df.merge(df_absent, on='catcode', how='left')
df['Work Code']=df['Work Code'].fillna('No Data')
df['Absent?']=df['Absent?'].fillna('TRUE')

# Update group
df=df.merge(df_group, on='group', how='left')

# Convert 'reg' to numeric (handle invalid strings as NaN)
#df['reg'] = pd.to_numeric(df['reg'], errors='coerce')
#df['ot1']=pd.to_numeric(df['ot1'], errors='coerce')
#df['ot2']=pd.to_numeric(df['ot2'], errors='coerce')
#df['ot3']=pd.to_numeric(df['ot3'], errors='coerce')

#calculate hours
df['Regular hours']=(df['reg']/3600).fillna(0)
df['OT1 (1.5)']=(df['ot1']/3600).fillna(0)
df['OT (2)']=(df['ot2']/3600).fillna(0)
df['OT3']=(df['ot3']/3600).fillna(0)

#calculate date
df['Date'] = df['Date'].apply(
    lambda x: pd.to_datetime(x, unit='s', origin='1900-01-01') if x!=0 else 'No Data'
)

df['Start Time'] = df['Start Time'].apply(
    lambda x: pd.to_datetime(x, unit='s', origin='1900-01-01') if x!=0 else 'No Data'
)

df['End Time'] = df['End Time'].apply(
    lambda x: pd.to_datetime(x, unit='s', origin='1900-01-01') if x!=0 else 'No Data'
)

#create full name
df['Full Name'] = (df['firstname'] + " " + df['lastname']).str.title()

#create Planned VS Unplanned Leave
# List of Work Codes considered as Unplanned
unplanned_codes = [
    'Personal Leave', 'Carers Leave - Paid', 'Leave Without Pay', 'No Data',
    'SICK LEAVE - PAID', 'Personal Leave No Cert', 'Compassionate Leave',
    'Unpaid Leave', 'SICK LEAVE - UNPAID', 'Unpaid Personal Leave','Personal Leave No Certificate'
]

# Create the new column
df['Planned VS Unplanned Leave'] = df.apply(
    lambda x: 'Work' if x['Absent?'] == False 
    else ('Unplanned' if x['Work Code'] in unplanned_codes 
          else 'Planned'),
    axis=1
)

# final columns
df=df[['ID','lastname',	'firstname','Full Name','Date',	'Start Time','End Time','Work Code','Absent?','Group','Regular hours','OT1 (1.5)','OT (2)',	'OT3','fulltime','permanent','Planned VS Unplanned Leave']]

# keep permanent and fulltime as string
#df['permanent'] = df['permanent'].astype(bool).map({True: 'TRUE', False: 'FALSE'})
#df['fulltime'] = df['fulltime'].astype(bool).map({True: 'TRUE', False: 'FALSE'})


# export
df.to_excel(r"C:\Users\141823\OneDrive - Etex Group\Documents\WFP Dashboard and Wages Model\Raw Timesheets\Final Timesheet.xlsx", index=False)

